# Newtork Tour of mice genetics

In [1]:
import numpy as np
import pandas as pd
import glob

### Loading the data

In [2]:
DATA_DIR = "../../Data/"

In [3]:
pheno = pd.read_csv(DATA_DIR+'Phenotype.txt', sep="\t", index_col="PhenoID")
pheno.head()

,B6D2F1,D2B6F1,C57BL.6J,DBA.2J,BXD1,BXD2,BXD5,BXD6,BXD8,BXD9,...,BXD90,BXD91,BXD93,BXD94,BXD95,BXD98,BXD99,BXD100,BXD101,BXD102
PhenoID,,,,,,,,,,,,,,,,,,,,,
X3063,NaN,NaN,1254.090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,707.929,1481.930,NaN,577.571,1971.000,NaN
X3352,NaN,NaN,724.205,NaN,NaN,NaN,NaN,700.643,1304.79,NaN,...,NaN,NaN,NaN,NaN,525.393,765.048,NaN,579.286,691.286,NaN
X1093,NaN,NaN,887.500,507.40,NaN,NaN,NaN,NaN,NaN,NaN,...,366.5,NaN,NaN,NaN,472.640,1060.340,1032.26,646.000,978.000,NaN
X2256,NaN,NaN,607.708,460.07,NaN,NaN,NaN,NaN,NaN,NaN,...,236.4,NaN,NaN,NaN,426.500,788.400,965.60,563.900,1272.400,NaN
X830,NaN,NaN,35444.000,25325.00,NaN,NaN,NaN,NaN,NaN,NaN,...,21515.0,NaN,NaN,NaN,21898.000,38538.000,36348.00,26982.000,39328.000,NaN


In [4]:
mapBXD = pd.read_csv(DATA_DIR+'map_BXD.txt', sep="\t", index_col="SNP")
mapBXD.head()

,Chr,Pos
SNP,,
rs31443144,1,3.010274
rs6269442,1,3.492195
rs32285189,1,3.511204
rs258367496,1,3.659804
rs32430919,1,3.777023


In [50]:
phenoID = pd.read_csv(DATA_DIR+'Phenotypes_id_aligner.txt', sep="\t", index_col = "PhenoID", encoding = "ISO-8859-1")
phenoID.head()

,PhenoID.1,RecordID,Phenotype,Shown_pheno,Category,Category_ID,color,Phenotype_post,Phenotype_pre,Authors,Senior.Author,Abbreviation,Quantitive.trait,Tissues,Diet,Strains
PhenoID,,,,,,,,,,,,,,,,
X3063,3063,12890,"Central nervous system, metabolism, behavior: ...","Activity of 13-week old females, total counts ...",Activity,1.0,1.0,"Central nervous system, metabolism, behavior: ...","Central nervous system, metabolism, behavior: ...","Andreux P, Williams EG, Koutnikova H, Houtkoop...",Auwerx_1,Activity of 13-week old females,Yes,muscle,NaN,19
X3352,3352,12960,"Central nervous system, metabolism, behavior: ...","Activity of 13-week old males, total counts of...",Activity,2.0,1.0,"Central nervous system, metabolism, behavior: ...","Central nervous system, metabolism, behavior: ...","Andreux P, Williams EG, Koutnikova H, Houtkoop...",Auwerx_1,Activity of 13-week old males,Yes,muscle,NaN,30
X1093,1093,15698,"Metabolism, behavior: Activity (ambulations) a...","Activity (ambulations) at night, metabolic cag...",Activity,3.0,1.0,"Metabolism, behavior: Activity (ambulations) a...",Trait_137_AUW-BXD,"Williams EG, Andreux P, Houtkooper R, Auwerx J",Auwerx_2,Activty (ambulations)_night_CD_AUC,Yes,muscle,NaN,42
X2256,2256,15699,"Metabolism, behavior: Activity (ambulations) a...","Activity (ambulations) at night, metabolic cag...",Activity,4.0,1.0,"Metabolism, behavior: Activity (ambulations) a...",Trait_138_AUW-BXD,"Williams EG, Andreux P, Houtkooper R, Auwerx J",Auwerx_2,Activty (ambulations)_night_HFD_AUC,Yes,muscle,HFD,40
X830,830,17629,"Metabolism, behavior: Activity (movement) at d...","Activity (movement) at dark phase (night) _CD,...",Activity,5.0,1.0,"Metabolism, behavior: Activity (movement) at d...","Metabolism, behavior: Activity (movement) at d...","Williams EG, Wu Y, Jha P, Dubuis S, Amariuta T...",Auwerx_2,Activity (movement) at_night_CD [AUC],Yes,muscle,NaN,45


In [6]:
geno = pd.read_csv(DATA_DIR+'Genotype_BXD.txt', sep="\t", index_col=["SNP"])
geno.head()

,B6D2F1,D2B6F1,C57BL.6J,DBA.2J,BXD1,BXD2,BXD5,BXD6,BXD8,BXD9,...,BXD90,BXD91,BXD93,BXD94,BXD95,BXD98,BXD99,BXD100,BXD101,BXD102
SNP,,,,,,,,,,,,,,,,,,,,,
rs31443144,0,0,-1,1,-1,-1,1,1,1,-1,...,-1,1,-1,1,1,-1,-1,-1,-1,-1
rs6269442,0,0,-1,1,-1,-1,1,1,1,-1,...,-1,1,-1,1,1,-1,-1,-1,-1,-1
rs32285189,0,0,-1,1,-1,-1,1,1,1,-1,...,-1,1,-1,1,1,-1,-1,-1,-1,-1
rs258367496,0,0,-1,1,-1,-1,1,1,1,-1,...,-1,1,-1,1,1,-1,-1,-1,-1,-1
rs32430919,0,0,-1,1,-1,-1,1,1,1,-1,...,-1,1,-1,1,1,-1,-1,-1,-1,-1


In [14]:
filenames = glob.glob(DATA_DIR+"expression data" + "/*.txt")

dfs = [pd.read_csv(f, sep="\t") for f in filenames]

expression = pd.concat(dfs, ignore_index=True, sort=False)

In [20]:
expression = expression.set_index(["gene"])
expression.head()

,C57BL.6J,DBA.2J,BXD68,BXD43,BXD44,BXD45,BXD48,BXD49,BXD50,BXD51,...,BXD54,BXD59,BXD72,BXD76,BXD78,BXD86,BXD88,BXD91,BXD93,BXD94
gene,,,,,,,,,,,,,,,,,,,,,
Nmt1_10338077,11.150,11.440,11.280,11.702,11.654,11.404,11.453,11.169,11.652,11.492,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Psmb5_10338095,12.906,13.125,13.144,13.099,13.071,13.048,12.825,12.993,12.915,13.142,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Hdac5_10338098,9.060,9.563,9.059,9.425,9.245,8.998,8.836,9.120,9.343,9.014,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dlg1_10338109,7.226,6.781,3.735,5.143,4.652,7.637,8.119,7.908,7.954,8.832,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wars_10338116,10.601,10.554,10.742,10.298,10.520,10.795,10.247,10.807,10.572,10.366,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Data Exploration

(5092, 0)

In [51]:
pheno.shape, phenoID.shape,expression.shape,mapBXD.shape,geno.shape

((5092, 97), (5092, 16), (1214846, 97), (7324, 2), (7324, 97))

In [40]:
phenoID["Category"].unique()

array(['Activity', 'Anxiety', 'Drug response', 'Hematology', 'Immune',
       'Metabolism', 'Metabolites', 'Microbiome', 'Morphology',
       'Musculoskeletal', 'Nervous system', 'Other'], dtype=object)

In [75]:
phenoID[phenoID["Shown_pheno"].str.contains("body weight")].shape

(120, 16)

In [83]:
phenoID.loc[phenoID["Category"] == "Microbiome","Shown_pheno"].unique()

array(['Microbiome, Actinobacteria (class) frequency assessed by 16S rRNA sequencing of fecal pellets from young adult males and females [residuals, log10 of fraction]',
       'Microbiome, Actinobacteria (phylum) frequency assessed by 16S rRNA sequencing of fecal pellets from young adult males and females [residuals, log10 of fraction]',
       'Microbiome, Actinobacteridae (subclass) frequency assessed by 16S rRNA sequencing of fecal pellets from young adult males and females [residuals, log10 of fraction]',
       'Microbiome, Actinomycetales (order) frequency assessed by 16S rRNA sequencing of fecal pellets from young adult males and females [residuals, log10 of fraction]',
       'Microbiome, Aerococcaceae (family) frequency assessed by 16S rRNA sequencing of fecal pellets from young adult males and females [residuals, log10 of fraction]',
       'Microbiome, Aerococcus (genus) frequency assessed by 16S rRNA sequencing of fecal pellets from young adult males and females [residuals

### Graphs construction

In [21]:
import networkx as nx
from scipy.spatial.distance import pdist, squareform
from matplotlib import pyplot as plt

ideas:

<div class="alert alert-block alert-info"> Build a graph with nodes == phenotypes and edges == gene participation probability. GNN --> predict edge weight

<div class="alert alert-block alert-info"> Build a graph with nodes == ... check microbiome levels (phenotype) in function of protein expression in gastrointestinal

<div class="alert alert-block alert-info"> Build a graph with nodes == ... check interaction between SNP and protein expression